In [1]:
from geemap import timelapse

In [2]:
## For allowing timeseries from GUI

In [3]:
timeseries_defs = [x for x in dir(timelapse) if '_timeseries' in x and '_legacy' not in x and 'create_' not in x]
timeseries_defs

['dynamic_world_timeseries',
 'goes_fire_timeseries',
 'goes_timeseries',
 'landsat_timeseries',
 'modis_ocean_color_timeseries',
 'modis_timeseries',
 'naip_timeseries',
 'sentinel1_timeseries',
 'sentinel2_timeseries']

In [4]:
from collections import defaultdict, Counter

def get_defs_variables(module='timelapse', alike=['_timelapse'], notalike=['_legacy']):
    defs = [x for x in dir(eval(module)) 
                if all(y in x for y in alike) 
                and not any(y in x for y in notalike)]
    varnames = defaultdict(list)
    counter = Counter()
        
    for x in defs:
        for k in eval(f'{module}.{x}.__code__.co_varnames'):
            varnames[k].append(x.split('_', 1)[0])
            counter[k] += 1
    return defs, counter, varnames

defs, counter, varnames = get_defs_variables(module='timelapse', alike=['_timelapse'], notalike=['_legacy'])

In [5]:
defs, len(defs)

(['create_timelapse',
  'goes_fire_timelapse',
  'goes_timelapse',
  'landsat_timelapse',
  'modis_ndvi_timelapse',
  'modis_ocean_color_timelapse',
  'naip_timelapse',
  'sentinel1_timelapse',
  'sentinel2_timelapse'],
 9)

In [6]:
counter

Counter({'collection': 4,
         'start_date': 9,
         'end_date': 9,
         'region': 1,
         'bands': 7,
         'frequency': 6,
         'reducer': 4,
         'date_format': 8,
         'out_gif': 9,
         'palette': 4,
         'vis_params': 6,
         'dimensions': 9,
         'frames_per_second': 6,
         'crs': 9,
         'overlay_data': 9,
         'overlay_color': 9,
         'overlay_width': 9,
         'overlay_opacity': 9,
         'title': 6,
         'title_xy': 6,
         'add_text': 6,
         'text_xy': 6,
         'text_sequence': 9,
         'font_type': 9,
         'font_size': 9,
         'font_color': 9,
         'add_progress_bar': 9,
         'progress_bar_color': 9,
         'progress_bar_height': 9,
         'add_colorbar': 3,
         'colorbar_width': 3,
         'colorbar_height': 3,
         'colorbar_label': 3,
         'colorbar_label_size': 3,
         'colorbar_label_weight': 3,
         'colorbar_tick_size': 3,
         'colorb

In [7]:
varnames

defaultdict(list,
            {'collection': ['create', 'modis', 'naip', 'sentinel1'],
             'start_date': ['create',
              'goes',
              'goes',
              'landsat',
              'modis',
              'modis',
              'naip',
              'sentinel1',
              'sentinel2'],
             'end_date': ['create',
              'goes',
              'goes',
              'landsat',
              'modis',
              'modis',
              'naip',
              'sentinel1',
              'sentinel2'],
             'region': ['create'],
             'bands': ['create',
              'goes',
              'landsat',
              'modis',
              'naip',
              'sentinel1',
              'sentinel2'],
             'frequency': ['create',
              'landsat',
              'modis',
              'naip',
              'sentinel1',
              'sentinel2'],
             'reducer': ['create', 'modis', 'naip', 'sentinel1'],
            